## Classical SDP relaxation of OPF
As a first step, we present a simple implementation of the classical SDP formulation of the Optimnal Power Flow problem using CVXPY.

### Setup
We start by importing pandapower for the test cases;
CVXPY, an open source Python-embedded modeling language for convex optimization problems;
MOSEK, a large scale optimization software. 

In [1]:
import pandapower.networks as nw
import pandapower as pp
import numpy as np
import cvxpy as cp
from scipy.sparse import csr_matrix
import mosek
import plotly.express as px

### Load Network Data
Now,let's make a function that loads the IEEE test cases from Pandapower.

In [2]:
# Select IEEE test case
def load_network(case):
    if case == 5:
        return nw.case5()
    if case == 14:
        return nw.case14()
    elif case == 30:
        return nw.case30()
    elif case == 57:
        return nw.case57()
    elif case == 118:
        return nw.case118()
    elif case == 300:
        return nw.case300()
    elif case == 'GB':
        return nw.GBnetwork()
    else:
        raise ValueError("Unsupported test case")

In [24]:
# Load the chosen IEEE test case
case_number = 30 # Change to 30, 57, or 118 for larger cases
net = load_network(case_number)

pp.runpp(net)  # Run power flow to initialize values

/home/catz0449/VSDP/env/lib/python3.12/site-packages/pandapower/io_utils.py:450: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/catz0449/VSDP/env/lib/python3.12/site-packages/pandapower/io_utils.py:450: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/catz0449/VSDP/env/lib/python3.12/site-packages/pandapower/io_utils.py:450: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/catz

In [25]:
net.line

,c_nf_per_km,df,from_bus,g_us_per_km,in_service,length_km,max_i_ka,max_loading_percent,name,parallel,r_ohm_per_km,std_type,to_bus,type,x_ohm_per_km
0,436.639076,1.0,0,0.0,True,1.0,0.555967,100.0,None,1,3.6450,None,1,ol,10.9350
1,291.092717,1.0,0,0.0,True,1.0,0.555967,100.0,None,1,9.1125,None,2,ol,34.6275
2,291.092717,1.0,1,0.0,True,1.0,0.277983,100.0,None,1,10.9350,None,3,ol,30.9825
3,0.000000,1.0,2,0.0,True,1.0,0.555967,100.0,None,1,1.8225,None,3,ol,7.2900
4,291.092717,1.0,1,0.0,True,1.0,0.555967,100.0,None,1,9.1125,None,4,ol,36.4500
5,291.092717,1.0,1,0.0,True,1.0,0.277983,100.0,None,1,10.9350,None,5,ol,32.8050
6,0.000000,1.0,3,0.0,True,1.0,0.384900,100.0,None,1,1.8225,None,5,ol,7.2900
7,145.546359,1.0,4,0.0,True,1.0,0.299367,100.0,None,1,9.1125,None,6,ol,21.8700
8,145.546359,1.0,5,0.0,True,1.0,0.555967,100.0,None,1,5.4675,None,6,ol,14.5800
9,0.000000,1.0,5,0.0,True,1.0,0.136853,100.0,None,1,1.8225,None,7,ol,7.2900


When a power flow is carried out, the element based grid model is translated into a bus-branch model. That bus-branch model is stored in a data structure that is based on the PYPOWER/MATPOWER casefile (with some extensions). This ppc can be accessed after power flow using net._ppc
We will get the bus admittance matrix using exactly that and extract all the parameters we need.

In [26]:

#make sure all units are consistent (convert everything to per unit)
Sbase = net.sn_mva # unit value for power MVA
Vbase = 135 # unit value for voltage kV
Zbase = Vbase**2 / Sbase # base impedance in Ohms
Ybase = 1 / Zbase # base admittance in S

#Now, we extract network parameters
Ybus = net._ppc["internal"]["Ybus"].todense() # Bus admittance matrix (Already in per unit. We need todense since the matrices are stored as sparcse matrices, but dense are easier to work with)
n = len(net.bus)  # Number of buses
# slack_bus = net.gen.bus.values[0]  # Index of the slack bus

#we need line information to add line flow constraints
# Extract line connections as (from_bus, to_bus) tuples
lines = list(zip(net.line['from_bus'].astype(int), net.line['to_bus'].astype(int)))

# Calculate series admittance for each line (in per unit)
y_lines = {}
for idx, row in net.line.iterrows():
    l = int(row['from_bus'])
    m = int(row['to_bus'])
    r = row['r_ohm_per_km'] * row['length_km']
    x = row['x_ohm_per_km'] * row['length_km']
    z = r + 1j * x
    y_lines[(l, m)] = 1 / z if z != 0 else 0

###################### Double check the shunt ######################

f = 50  # Hz, change if your system is US, which uses 60 Hz
y_shunt = {}
for idx, row in net.line.iterrows():
    l = int(row['from_bus'])
    m = int(row['to_bus'])
    c_nf = row['c_nf_per_km'] * row['length_km']  # nF
    c_f = c_nf * 1e-9  # convert nF to F
    y_shunt_lm = 1j * 2 * np.pi * f * c_f * Ybase  # in per unit
    y_shunt[(l, m)] = y_shunt_lm

##################### Double check s_max ####################################

# Define s_max for each line using max_i_ka and Vbase
s_max = {}
for idx, row in net.line.iterrows():
    l = int(row['from_bus'])
    m = int(row['to_bus'])
    Imax = row['max_i_ka']  # in kA
    # Smax in MVA (assuming Vbase is in kV)
    s_max[(l, m)] = np.sqrt(3) * Vbase * Imax

######################################################################

# Extract generator information for generation constraints
gen_buses = net.gen["bus"].values #index of the generators.
min_p_mw = net.gen["min_p_mw"].values #minimal real power
max_p_mw = net.gen["max_p_mw"].values
min_q_mw = net.gen["min_q_mvar"].values #minimal reactive power
max_q_mw = net.gen["max_q_mvar"].values

# Extract load buses for generation constraints
load_buses = net.load["bus"].values #index of the load buses.
#load active and reactive power for load
p_load = np.zeros(n)
q_load = np.zeros(n)
for i,row in net.load.iterrows():
    p_load[row['bus']] = -row['p_mw'] #negative sign because pandapower defines load as positive, but we need it as negative for consumption
    q_load[row['bus']] = -row['q_mvar']

# Voltage limits
min_v_pu = net.bus["min_vm_pu"].values ** 2  # Squared for SDP
max_v_pu = net.bus["max_vm_pu"].values ** 2

# Slack bus voltage
vm_slack = net.ext_grid['vm_pu'][0] #initial voltage of the slack

####### Classify the buses into gen, load, and slack buses ##########
# Get sets of generator and load buses
gen_buses_set = set(net.gen["bus"].astype(int).tolist())
load_buses_set = set(net.load["bus"].astype(int).tolist())

# Slack bus
slack_bus = int(net.ext_grid["bus"].iloc[0])

# Classify buses
bus_roles = {}
for k in range(n):
    if k == slack_bus:
        bus_roles[k] = "slack"
    elif k in gen_buses_set and k in load_buses_set:
        bus_roles[k] = "gen+load"
    elif k in gen_buses_set:
        bus_roles[k] = "gen-only"
    elif k in load_buses_set:
        bus_roles[k] = "load-only"
    else:
        bus_roles[k] = "pass-through"  # e.g., junctions, transformers
#######################################################################

# Extract generator cost coefficients from net.poly_cost
cost = net.poly_cost.drop([0])
c2 = cost["cp2_eur_per_mw2"].values   # Quadratic cost coefficients
c1 = cost["cp1_eur_per_mw"].values   # Linear cost coefficients
c0 = cost["cp0_eur"].values         # Constant cost coefficients

# Ensure the cost coefficients have the correct size
if len(c2) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
elif len(c1) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
elif len(c0) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
else:
    print("The number of generators matches the number of cost coefficients")

for k in range(n):
    print(f"Bus {k}: {bus_roles[k]}")

The number of generators matches the number of cost coefficients
Bus 0: slack
Bus 1: gen+load
Bus 2: load-only
Bus 3: load-only
Bus 4: pass-through
Bus 5: pass-through
Bus 6: load-only
Bus 7: load-only
Bus 8: pass-through
Bus 9: load-only
Bus 10: pass-through
Bus 11: load-only
Bus 12: gen-only
Bus 13: load-only
Bus 14: load-only
Bus 15: load-only
Bus 16: load-only
Bus 17: load-only
Bus 18: load-only
Bus 19: load-only
Bus 20: load-only
Bus 21: gen-only
Bus 22: gen+load
Bus 23: load-only
Bus 24: pass-through
Bus 25: load-only
Bus 26: gen-only
Bus 27: pass-through
Bus 28: load-only
Bus 29: load-only


In [43]:
print(min_p_mw)

[0. 0. 0. 0. 0.]


### Define Variables and the objective function

The objective function of standard Optimal Power Flow (OPF) problem is:
$$\min \space \sum_{j \in G} f_j(P_{j})$$
where $f_j(P_{j})$ is the cost function of choice. Traditionally, the following cost function has been used:
$$\min \space \sum_{j \in G}( c_{j2}P_j^2 + c_{j1}P_j + c_{jo})$$
For each bus $j$ in the grid $G$:
$P_j = power \space generation, \space c_{j2} = quadratic \space cost \space  coefficient, \space c_{j1} = linear \space cost \space  coefficient, \space c_{j0} = constant \space cost.$

The objective function is then subjected to a set of constraints.

Semidefinite Programming (SDP) is one of the conic form problems, which all have a linear objective function and convex inequality/equality constraints (that could be quadratic). SDP in particular tackles the cone of positive semidefinite n x n matrices, W. The SDP Standard Form objective function is as follows:

$$min \space \space tr(CW)$$
subjected to $W \succcurlyeq 0$ and other linear constraints, which are

**Semidefinite positivity**
$$W\succcurlyeq 0$$

**Active Power Flow**
$$p_{min, j} \le tr (\Phi_jW_B) \le p_{max, j}$$
where $\Phi_j = \frac{1}{2}(Y^H_j + Y_j)$

**Reactive Power Flow**
$$q_{min, j} \le tr (\Psi_jW_B) \le q_{max, j}$$
where $\Psi_j = \frac{1}{2i}(Y^H_j + Y_j)$

**Voltage Magnitude Constraints**
$$v_{min, j} \le tr (J_jW_B) \le v_{max, j}$$
where J is the Hermitian matrix with a single 1 in the (j,j)th entry and zero everywhere else.

To transform the quadratic objective function into SDP, we can do the following:

#### Attempt 1

Let $$C_j := \begin{bmatrix}c_{j2} & c_{j1}/2 \\c_{j1}/2 & c_{j0}\end{bmatrix},\space X_j := \begin{bmatrix}a_j & P_j \\P_j & 1\end{bmatrix}$$
where $a_j$ is a new variable meant to represent $P_j^2$. Instead of enforcing an equality, we will relax it into a matrix inequality. We add the following constraint:
$$\begin{bmatrix}a_j & P_j \\P_j & 1\end{bmatrix} \succcurlyeq 0 $$
By Schur’s complement condition, the constraint above is equivalent to
$$P^2_j \le a_j$$
As we optimize, the solver will push the above inequality toward equality. Now, we can formulate the objective function as:
$$min \space \sum_{j \in G} tr(C_jX_j) $$
which is equivalent to $$\min \space \sum_{j \in G}( c_{j2}P_j^2 + c_{j1}P_j + c_{jo})$$

In [27]:
# # Define an arbitrary n x n Hermitian W (Directly enforcing W= V V^+ will require X to be rank-1, which is too strict. We use the power balance, PSD and voltage constraints to guide this arbitrary X to X = VV^+)
# W = cp.Variable((n, n), hermitian=True)

# # Define phi and psi matrices for each bus as the constraints
# Phi = [] # Hermitian part of Y_j
# Psi = [] # Skewed Hermitian part of Y_j
# J = [] # Hermitian matrix with a single 1 in the (j,j)th entry and zero everywhere else.
# for j in range(n):
#     Y_j = np.zeros_like(Ybus,dtype=complex) #Y_j is the admittance matrix with only the j-th row, everything else is zero
#     Y_j[j,:] = Ybus[j,:]
#     Y_j_H = np.conj(Y_j).T #Hermitian of Y_j
    
#     J_j= np.zeros_like(Ybus,dtype=complex)
#     J_j[j,j] = 1

#     Phi_j = 1/2 * (Y_j_H + Y_j) 
#     Psi_j = 1/(2j) * (Y_j_H - Y_j)

#     Phi.append(Phi_j) 
#     Psi.append(Psi_j)
#     J.append(J_j)

# Objective Function Formulation in Standard SDP Form
################################################################################
# # # Attempt 0 (Objective function in polynomial form)


# # Define the objective function, which is the sum of the costs of all generators
# objective_terms = cp.sum([
#     c2[i] * (cp.real(cp.trace(Phi[gen_buses[i]] @ W))*Sbase)**2 + 
#     c1[i] * cp.real(cp.trace(Phi[gen_buses[i]] @ W))*Sbase for i in range(len(gen_buses))])
# objective = cp.Minimize(objective_terms)

# ############################################################################

# # Attempt 1 (method above)
# # Initialize 
# C = []
# X = []

# for i in range(len(gen_buses)):
#     # Real power generation at generator i
#     P_j = cp.real(cp.trace(Phi[gen_buses[i]] @ W)) * Sbase
#     # We want a = P^2, and we will add a psd constraint later to make it happen
#     a = cp.Variable()

#     # Define the cost matrix for generator i
#     C_i = cp.Constant(np.array([[c2[i], c1[i]/2], [c1[i]/2, c0[i]]]))
#     C.append(C_i)

#     # Define the variable matrix for generator i
#     X_i = cp.vstack([
#         cp.hstack([a, P_j]),
#         cp.hstack([P_j, 1])
#     ])
#     X.append(X_i)

# objective_terms = cp.sum([
#     cp.trace(C[i] @ X[i]) for i in range(len(gen_buses))
# ])

# objective = cp.Minimize(objective_terms)



##############################################################################


##############################################################################

# # Constraints
# constraints = []

# for j in range(n):
#     if j == 0:
#         # For the slack bus, we fix the voltage magnitude
#         constraints.append(cp.abs(cp.real(cp.trace(J[j] @ W))-(vm_slack)**2) <= 1e-5)
#     else:

#         # Voltage magnitude constraints
#         constraints.append(min_v_pu[j]<=cp.real(cp.trace(J[j] @ W)))
#         constraints.append(cp.real(cp.trace(J[j] @ W))<=max_v_pu[j])
    
#     # #Change of variables
#     # constraints.append(a[j] == cp.real(cp.trace(Phi[j] @ W)))*Sbase

#     # Power balance equations (real and reactive)
#     Pin_j = cp.trace(Phi[j]@W)*Sbase
#     Qin_j = cp.trace(Psi[j]@W)*Sbase

#     # If bus is a generator bus, enforce generation limits
#     if j in gen_buses:
#         idx = np.where(gen_buses == j)[0][0]  # Get generator index
#         constraints.append(min_p_mw[idx] <= cp.real(Pin_j))
#         constraints.append(cp.real(Pin_j) <= max_p_mw[idx])
#         constraints.append(min_q_mw[idx] <= cp.real(Qin_j))
#         constraints.append(cp.real(Qin_j) <= max_q_mw[idx])
#         # #add psd constraint to make a = P^2
#         # constraints.append(X[idx] >> 0)
#         #  # For generator buses, enforce power balance constraints
#         # constraints.append(cp.real(Pin_j) == p_load[j])
#         # constraints.append(cp.real(Qin_j) == q_load[j])
#     else:
#         # For non-generator buses, enforce power balance constraints
#         constraints.append(cp.real(Pin_j) == p_load[j])
#         constraints.append(cp.real(Qin_j) == q_load[j])

# # Positive semidefinite constraint
# constraints.append(W >> 0)

### Attempt 2 (Optimization 3 in J.Lavaei and S.H.Low, 2012)

In [44]:


############################## Initialize ###################################
# Define our problem matrix, W. It is 2n x 2n, where the first n rows and columns correspond to the real part of the voltage, and the second n rows and columns correspond to the imaginary part of the voltage.
W = cp.Variable((2*n, 2*n), PSD=True)

# Initialize the problem matrices
Y = [] # Hermitian part of Y_j
Y_bar = [] # Skewed Hermitian part of Y_j
M = []

for k in range(n):
    e_k = np.zeros((n, 1))
    e_k[k] = 1
    Yk = e_k @ e_k.T.conj() @ Ybus #Y_K is the admittance matrix with only the (k,k) entry, everything else is zero
    
    #Y_k is the Hermitian part of Y_k
    Y_k = 1/2 * np.block([
        [np.real(Yk + Yk.T),     np.imag(Yk.T - Yk)],
        [np.imag(Yk - Yk.T),     np.real(Yk + Yk.T)]
    ])
    #Y_kbar is the skewed Hermitian part of Y_k
    Y_kbar = -1/2 * np.block([
        [np.imag(Yk + Yk.T),    np.real(Yk - Yk.T)],
        [np.real(Yk.T - Yk),    np.imag(Yk + Yk.T)]
    ])
    #M_k is the matrix that will be used to enforce the voltage constraints
    M_k = np.block([
        [e_k @ e_k.T,       np.zeros((n, n))],
        [np.zeros((n, n)), e_k @ e_k.T]
    ])

    Y.append(Y_k)
    Y_bar.append(Y_kbar)
    M.append(M_k)

# Create the matrices for the line constraints
M_lm = {}
Y_lm = {}
Y_lmbar = {}
for (l, m) in lines:  # lines is a list of (l, m) tuples

    e_l = np.zeros((n, 1))
    e_m = np.zeros((n, 1))
    e_l[l] = 1
    e_m[m] = 1
    delta = e_l - e_m
    Mlm = np.block([
        [delta @ delta.T,          np.zeros((n, n))],
        [np.zeros((n, n)), delta @ delta.T]
    ])
    M_lm[(l, m)] = Mlm

    # Assume y_lm and y_shunt_lm are given for line (l,m)
    y_lm = y_lines[(l, m)]         # complex scalar
    y_shunt_lm = y_shunt[(l, m)]  # complex scalar

    Ylm = (y_shunt_lm + y_lm) * (e_l @ e_l.T) - y_lm * (e_l @ e_m.T)

    # Real-valued lifting
    Re_Y = np.real(Ylm + Ylm.T)
    Im_Y = np.imag(Ylm.T - Ylm)
    Re_Y_skew = np.real(Ylm - Ylm.T)
    Im_Y_sum = np.imag(Ylm + Ylm.T)

    Y_lm[(l, m)] = 1/2 * np.block([
        [Re_Y,      Im_Y],
        [-Im_Y,     Re_Y]
    ])
    Y_lmbar[(l, m)] = -1/2 * np.block([
        [Im_Y_sum,      Re_Y_skew],
        [-Re_Y_skew,    Im_Y_sum]
    ])

############################## Objective Function #############################


alpha = cp.Variable(len(gen_buses))
objective = cp.Minimize(cp.sum(alpha))

    
############################## Constraints ####################################
constraints = []

for k in range(n):

    role = bus_roles[k]

    # For the slack bus, we fix the voltage magnitude since it is not adjustable
    if role == "slack":
        constraints.append(cp.trace(M[k] @ W) == vm_slack**2)

    # Power calculations
    Pin_k = cp.trace(Y[k] @ W) * Sbase
    Qin_k = cp.trace(Y_bar[k] @ W) * Sbase
        
    if role == "gen+load":
        idx = np.where(gen_buses == k)[0][0]
        Pg_min = min_p_mw[idx] + p_load[k] # we use plus load here since load is negative
        print(f"Pg_min: {Pg_min}, p_load: {p_load[k]}")
        Pg_max = max_p_mw[idx] + p_load[k]
        Qg_min = min_q_mw[idx] + q_load[k]
        Qg_max = max_q_mw[idx] + q_load[k]
        constraints.append(Pg_min <= Pin_k)
        constraints.append(Pin_k <= Pg_max)
        constraints.append(Qg_min <= Qin_k)
        constraints.append(Qin_k <= Qg_max)

    elif role == "gen-only":
        idx = np.where(gen_buses == k)[0][0]
        constraints.append(min_p_mw[idx] <= Pin_k) 
        constraints.append(Pin_k <= max_p_mw[idx])
        constraints.append(min_q_mw[idx] <= Qin_k)
        constraints.append(Qin_k <= max_q_mw[idx])
        
    elif role == "load-only":
        constraints.append(Pin_k == p_load[k]) 
        constraints.append(Qin_k == q_load[k])
    
     # Voltage constraints (except slack)
    if role != "slack":
        constraints.append(min_v_pu[k] <= cp.trace(M[k] @ W))
        constraints.append(cp.trace(M[k] @ W) <= max_v_pu[k])


# Positive semidefinite constraint
constraints.append(W >> 0)

# Line Flow Constraints
for (l, m) in lines:
    Plm = cp.trace(Y_lm[(l, m)] @ W) * Sbase
    Qlm = cp.trace(Y_lmbar[(l, m)] @ W) * Sbase
    Smax = s_max[(l, m)]

    S_constr = cp.bmat([
        [-Smax**2, Plm, Qlm],
        [Plm, -1, 0],
        [Qlm, 0, -1]
    ])
    constraints.append(S_constr << 0)

# Schur Complement cost constraints
for idx, k in enumerate(gen_buses):
    Pin_k = cp.trace(Y[k] @ W) * Sbase
    Pgen_k = Pin_k + p_load[k]  # Total generation

    c2_k = float(c2[idx])
    c1_k = float(c1[idx])
    c0_k = float(c0[idx])
    Pd_k = float(p_load[k])

    a_k = c0_k + c1_k * Pd_k
    b_k = float(np.sqrt(c2_k)) * Pd_k

    top_left = c1_k * Pgen_k - alpha[idx] + a_k
    top_right = float(np.sqrt(c2_k)) * Pgen_k + b_k

    Schur = cp.bmat([
        [top_left, top_right],
        [top_right, -1]
    ])
    constraints.append(Schur << 0)


Pg_min: -21.7, p_load: -21.7
Pg_min: -3.2, p_load: -3.2


### Solve using cvxpy's SDP solver

In [42]:
# Solve the SDP
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)

# Print constraints
print("p min (MW):", min_p_mw)
print("p max:(MW)", max_p_mw)
print("q min (MW):", min_q_mw)
print("q max (MW):", max_q_mw)

# Print results
print("Problem Status:", prob.status)
print("Optimal cost EUR:", prob.value)
P_g_sdp = [cp.real(cp.trace(Y[k]@ W)).value * Sbase for k in gen_buses]
print("Real power generation in MW (SDP OPF):", P_g_sdp)
Q_g_sdp = [cp.imag(cp.trace(Y_bar[k] @ W)).value * Sbase for k in gen_buses]
print("Reactive power generation in MW (SDP OPF):", Q_g_sdp)

p min (MW): [0. 0. 0. 0. 0.]
p max:(MW) [80. 50. 55. 30. 40.]
q min (MW): [-20. -15. -15. -10. -15.]
q max (MW): [60.  62.5 48.7 40.  44.7]
Problem Status: optimal
Optimal cost EUR: -70.24599999999943
Real power generation in MW (SDP OPF): [-6.5999999950051125, -4.884981308350689e-13, 4.718447854656915e-14, -3.1999999999999917, 4.3368086899420177e-16]
Reactive power generation in MW (SDP OPF): [0.0, 0.0, 0.0, 0.0, 0.0]


In [39]:
print("Alpha values:", alpha.value)
print("Actual generation (MW):", [(cp.trace(Y[k] @ W).value * Sbase + p_load[k]) for k in gen_buses])
print("Cost coefficients c1:", c1)
print("Cost coefficients c2:", c2)

Alpha values: [-43.75    0.      0.    -26.496   0.   ]
Actual generation (MW): [-28.299999995005113, -4.884981308350689e-13, 4.718447854656915e-14, -6.3999999999999915, 4.3368086899420177e-16]
Cost coefficients c1: [1.75 1.   3.25 3.   3.  ]
Cost coefficients c2: [0.0175  0.0625  0.00834 0.025   0.025  ]


In [40]:
print("Load demand p_load:", p_load)
print("Total system load:", np.sum(p_load))

Load demand p_load: [  0.  -21.7  -2.4  -7.6   0.    0.  -22.8 -30.    0.   -5.8   0.  -11.2
   0.   -6.2  -8.2  -3.5  -9.   -3.2  -9.5  -2.2 -17.5   0.   -3.2  -8.7
   0.   -3.5   0.    0.   -2.4 -10.6]
Total system load: -189.2


### Check whether the solution is physically feasible
The semidefinite relaxation makes OPF convex, but also opens the door to:

1. Inexact solutions (i.e., WW is not rank-1)

2. Unrealistically cheap solutions that cannot occur in real networks.

Let's check our solution.

In [30]:
#Check the rank of W
eigvals = np.linalg.eigvalsh(W.value)
rank_W = np.sum(eigvals > 1e-6)  # numerical rank threshold
print("Rank of W:", rank_W)

# Check if the solution is feasible
for i in range(len(P_g_sdp)):
    if P_g_sdp[i] < min_p_mw[i]:
        print(f"Generation at bus {i} is below the minimum limit.")
    elif i > max_p_mw[i]:
        print(f"Generation at bus {i} is above the maximum limit.")

# Check if the voltage magnitudes are within limits
for i in range(n):
    if V_sdp[i]**2 < min_v_pu[i]:
        print(f"Voltage at bus {i} is below the minimum limit.")
    elif V_sdp[i]**2 > max_v_pu[i]:
        print(f"Voltage at bus {i} is above the maximum limit.")

# Check if the power balance equations are satisfied
for j in range(n):
    Pin_j = cp.trace(Phi[j] @ W).value * Sbase
    Qin_j = cp.trace(Psi[j] @ W).value * Sbase
    if j in gen_buses:
        idx = np.where(gen_buses == j)[0][0]  # Get generator index
        if Pin_j < min_p_mw[idx]:
            print(f"Power balance at bus {j} is below the minimum limit.")
        elif Pin_j > max_p_mw[idx]:
            print(f"Power balance at bus {j} is above the maximum limit.")
    # If bus is a load bus, check if the power balance is approximately equal to the load
    elif abs(Pin_j - p_load[j]) > 1e-5:
            print(f"Power balance at bus {j} does not match load.")



Rank of W: 60


NameError: name 'P_g_sdp' is not defined

### Benchmark using Classical OPF

In [31]:
pp.runopp(net)

#Print results
opf_cost = net.res_cost
print("OPF Cost ($/hr):", opf_cost)
P_g_opf = net.res_gen["p_mw"].values
print("OPF Real power generation (OPF):", P_g_opf)
Q_g_opf = net.res_gen["q_mvar"].values
print("OPF Reactive power generation (OPF):", Q_g_opf)
V_opf = net.res_bus["vm_pu"].values
print("OPF Voltage magnitudes (OPF):", V_opf)

/home/catz0449/VSDP/env/lib/python3.12/site-packages/pandapower/opf/validate_opf_input.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  net[element_type].controllable.fillna(True, inplace=True)
/home/catz0449/VSDP/env/lib/python3.12/site-packages/pandapower/opf/validate_opf_input.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on

OPF Cost ($/hr): 578.4862507022582
OPF Real power generation (OPF): [53.1027996559 22.3820387485 46.4594636979 15.4733376498 15.1251938751]
OPF Reactive power generation (OPF): [18.177124564  35.6965612024 22.6085000171  4.753350959  43.0477156893]
OPF Voltage magnitudes (OPF): [1.           1.0040292064 0.9991046544 0.9995256999 0.9954851976
 0.9951312557 0.9859487109 0.9839161988 1.0127310719 1.0220416194
 1.0127310719 1.0443418465 1.099003164  1.0319914924 1.0330046848
 1.0279272619 1.0186652162 1.0167993158 1.0106507586 1.0126129715
 1.0306383003 1.0370314969 1.0431011751 1.0330410003 1.049999722
 1.033043735  1.0689511092 1.0030235153 1.0499993112 1.0391128073]


### Comparison

In [32]:
print("SDP OPF Optimal cost (EUR):", prob.value)
print("OPF Cost (EUR):", opf_cost*0.87)

SDP OPF Optimal cost (EUR): -39.07842499999943
OPF Cost (EUR): 503.28303811096464


In [33]:
import plotly.graph_objects as go

# Generator power plot
gen_fig = go.Figure()
gen_fig.add_trace(go.Scatter(x=gen_buses, y=P_g_sdp, mode='markers+lines', name='SDP OPF Gen (MW)'))
gen_fig.add_trace(go.Scatter(x=gen_buses, y=P_g_opf, mode='markers+lines', name='OPF Gen (MW)'))
gen_fig.update_layout(title="Generator Real Power Comparison", xaxis_title="Generator Bus", yaxis_title="P (MW)")
gen_fig.show()

NameError: name 'P_g_sdp' is not defined

In [34]:
# Voltage Magnitude plot
gen_fig = go.Figure()
gen_fig.add_trace(go.Scatter(x=list(range(n)), y=V_sdp, mode='markers+lines', name='SDP OPF V (pu)'))
gen_fig.add_trace(go.Scatter(x=list(range(n)), y=V_opf, mode='markers+lines', name='OPF V (pu)'))
gen_fig.update_layout(title="Voltage Magnitude Comparison", xaxis_title="Bus", yaxis_title="V (pu)")
gen_fig.show()

NameError: name 'V_sdp' is not defined